In [30]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix

In [31]:
# Load the Excel file
df = pd.read_csv('preprocessed_image_train.csv')

# Check the first few rows to verify
df.head()

,Unnamed: 0,image_name,image_path,productid,imageid,prdtypecode,product_category
0,0,image_1000076039_product_580161.jpg,C:/Users/User/OneDrive - ingenium digital diag...,580161,1000076039,10,10 (Used books)
1,1,image_1000089455_product_348990858.jpg,C:/Users/User/OneDrive - ingenium digital diag...,348990858,1000089455,2280,2280 (Used Newspapers and magazines)
2,2,image_1000092894_product_353108104.jpg,C:/Users/User/OneDrive - ingenium digital diag...,353108104,1000092894,2403,"2403 (Books, comics, and magazines)"
3,3,image_1000093804_product_343306951.jpg,C:/Users/User/OneDrive - ingenium digital diag...,343306951,1000093804,1160,1160 (Board games and role-playing games)
4,4,image_1000095646_product_344209267.jpg,C:/Users/User/OneDrive - ingenium digital diag...,344209267,1000095646,1160,1160 (Board games and role-playing games)


In [32]:
df = df[['image_path', 'prdtypecode']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84909 entries, 0 to 84908
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_path   84909 non-null  object
 1   prdtypecode  84909 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [33]:
df.head()

,image_path,prdtypecode
0,C:/Users/User/OneDrive - ingenium digital diag...,10
1,C:/Users/User/OneDrive - ingenium digital diag...,2280
2,C:/Users/User/OneDrive - ingenium digital diag...,2403
3,C:/Users/User/OneDrive - ingenium digital diag...,1160
4,C:/Users/User/OneDrive - ingenium digital diag...,1160


In [34]:
count_df = df.groupby('prdtypecode').count()
count_df

,image_path
prdtypecode,
10,3116
40,2508
50,1681
60,832
1140,2671
1160,3953
1180,764
1280,4869
1281,2070


Split train and test

In [35]:
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['prdtypecode'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['prdtypecode'], random_state=42)

In [36]:
print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}, Test samples: {len(test_df)}")

Training samples: 67927, Validation samples: 8491, Test samples: 8491


In [37]:
# Get number of classes
num_classes = len(df['prdtypecode'].unique())

In [38]:
train_count = train_df.groupby('prdtypecode').count()
train_count

,image_path
prdtypecode,
10,2493
40,2006
50,1345
60,666
1140,2137
1160,3162
1180,611
1280,3895
1281,1656


In [39]:
# Define minority classes (below mean count)
mean_count = train_df['prdtypecode'].value_counts().mean()
minority_classes = train_df['prdtypecode'].value_counts()[train_df['prdtypecode'].value_counts() < mean_count].index
print(mean_count)
print(minority_classes)

2515.814814814815
Index([  10, 2705, 1140, 2582,   40, 2585, 1302, 1281,   50, 2462, 2905,   60,
       2220, 1301, 1940, 1180],
      dtype='int64', name='prdtypecode')


In [40]:
# === Step 2: Handle Class Imbalance with Targeted Augmentation ===
import os

# Augment and save images for minority classes
def augment_minority_classes(df, minority_classes, target_size=(224, 224), augment_count=3, output_dir="augmented_data"):
    os.makedirs(output_dir, exist_ok=True)  # Create directory if not exists
    
    augmenter = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    augmented_data = []
    
    for class_label in minority_classes:
        class_df = df[df['prdtypecode'] == class_label]
        for _, row in class_df.iterrows():
            img = Image.open(row['image_path']).resize(target_size)
            img_array = np.expand_dims(np.array(img), axis=0)
            i = 0
            for batch in augmenter.flow(img_array, batch_size=1):
                # Save augmented image
                file_name = f"aug_{class_label}{i}{os.path.basename(row['image_path'])}"
                file_path = os.path.join(output_dir, file_name)
                Image.fromarray((batch[0] * 255).astype('uint8')).save(file_path)
                
                # Append to augmented data
                augmented_data.append({'image_path': file_path, 'prdtypecode': class_label})
                i += 1
                if i >= augment_count:
                    break
    
    augmented_df = pd.DataFrame(augmented_data)
    return augmented_df

# Call the function and generate augmented images
output_dir = "augmented_images"  # Directory to save augmented images
augmented_df = augment_minority_classes(train_df, minority_classes, output_dir=output_dir)

# Combine augmented data with the original training set
train_df = pd.concat([train_df, augmented_df], ignore_index=True)


In [41]:
# Perform targeted augmentation for minority classes
##augmented_df = augment_minority_classes(train_df, minority_classes)

In [42]:
# Combine augmented data with the original training set
#train_df = pd.concat([train_df, augmented_df])
train_df.shape

(136825, 2)

In [43]:
train_df.to_csv('Image_Train_vgg16_new.csv')
val_df.to_csv('Image_Val_vgg16_new.csv')
test_df.to_csv('Image_Test_vgg16_new.csv')

In [44]:
train_count = train_df.groupby('prdtypecode').count()
train_count.sum()

image_path    136825
dtype: int64

In [45]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation and test data generators
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [46]:
# Convert prdtypecode to category
train_df['prdtypecode'] = train_df['prdtypecode'].astype('category')
val_df['prdtypecode'] = val_df['prdtypecode'].astype('category')
test_df['prdtypecode'] = test_df['prdtypecode'].astype('category')

# Convert category values to string for compatibility
train_df['prdtypecode'] = train_df['prdtypecode'].astype(str)
val_df['prdtypecode'] = val_df['prdtypecode'].astype(str)
test_df['prdtypecode'] = test_df['prdtypecode'].astype(str)

In [47]:
# Create generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='prdtypecode', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='prdtypecode',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='prdtypecode',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 136825 validated image filenames belonging to 27 classes.
Found 8491 validated image filenames belonging to 27 classes.
Found 8491 validated image filenames belonging to 27 classes.


In [48]:
# Calculate the number of batches in the train generator
num_samples = len(train_generator)  # Number of images in your training dataframe
batch_size = 32  # Your batch size

num_batches = num_samples // batch_size  # Integer division
if num_samples % batch_size != 0:
    num_batches += 1  # If there is a remainder, add one more batch

print(f"Number of batches in the training set: {num_batches}")

Number of batches in the training set: 134


In [49]:
# === Step 4: Compute Class Weights ===
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['prdtypecode'].astype(int)),
    y=train_df['prdtypecode'].astype(int)
)
class_weights = dict(enumerate(class_weights))

In [50]:
# === Step 5: Define the ResNet-50 Model ===
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base layers

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5, verbose=1)

In [ ]:
# === Step 6: Train the Model ===
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks = [reduce_lr]
)


c:\Users\User\OneDrive - ingenium digital diagnostics GmbH\Desktop\DataScientest\Rakuten project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12360s 3s/step - accuracy: 0.2914 - loss: 2.3374 - val_accuracy: 0.3883 - val_loss: 2.1297
Epoch 2/20
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12249s 3s/step - accuracy: 0.3308 - loss: 2.1921 - val_accuracy: 0.3896 - val_loss: 2.1262
Epoch 3/20
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12246s 3s/step - accuracy: 0.3329 - loss: 2.1912 - val_accuracy: 0.3904 - val_loss: 2.1225
Epoch 4/20
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12233s 3s/step - accuracy: 0.3321 - loss: 2.1925 - val_accuracy: 0.3912 - val_loss: 2.1204
Epoch 5/20
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12205s 3s/step - accuracy: 0.3305 - loss: 2.1888 - val_accuracy: 0.3918 - val_loss: 2.1182
Epoch 6/20
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12257s 3s/step - accuracy: 0.3301 - loss: 2.1869 - val_accuracy: 0.3912 - val_loss: 2.1160
Epoch 7/20
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12229s 3s/step - accuracy: 0.3344 - loss: 2.1799 - val_accuracy: 0.3928 - val_loss: 2.1134
Epoch 8/20
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12176s 3s/step - accuracy: 

In [ ]:
# Classification report and confusion matrix
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

266/266 ━━━━━━━━━━━━━━━━━━━━ 707s 3s/step


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.36      0.20      0.26       311
           1       0.34      0.22      0.27       267
           2       0.55      0.75      0.63       395
           3       0.00      0.00      0.00        77
           4       0.22      0.35      0.27       487
           5       0.27      0.01      0.03       207
           6       0.46      0.46      0.46       505
           7       0.00      0.00      0.00        81
           8       0.04      0.01      0.02       249
           9       0.49      0.16      0.24       324
          10       0.39      0.37      0.38       507
          11       0.53      0.68      0.59       431
          12       1.00      0.04      0.07        80
          13       0.34      0.24      0.28       499
          14       0.00      0.00      0.00        82
          15       0.39      0.61      0.48       476
          16       0.41      0.59      0.49       477
          17       0.00    

c:\Users\User\OneDrive - ingenium digital diagnostics GmbH\Desktop\DataScientest\Rakuten project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\OneDrive - ingenium digital diagnostics GmbH\Desktop\DataScientest\Rakuten project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\OneDrive - ingenium digital diagnostics GmbH\Desktop\DataScientest\Rakuten project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-d

In [ ]:
model.save('my_model_VGG16_reducelr_1e-5.keras')

In [ ]:
import json

# Save class indices from the train_generator
with open('class_indices_my_model_VGG16.json', 'w') as f:
    json.dump(train_generator.class_indices, f)

print("Class indices saved successfully!")

Class indices saved successfully!


In [ ]:
# === Step 6: Train the Model ===
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks = [reduce_lr]
)

Epoch 1/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12052s 3s/step - accuracy: 0.3285 - loss: 2.1991 - val_accuracy: 0.3812 - val_loss: 2.1411 - learning_rate: 0.0000e+00
Epoch 2/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12060s 3s/step - accuracy: 0.3283 - loss: 2.2015 - val_accuracy: 0.3812 - val_loss: 2.1411 - learning_rate: 0.0000e+00
Epoch 3/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12030s 3s/step - accuracy: 0.3274 - loss: 2.2009 - val_accuracy: 0.3812 - val_loss: 2.1411 - learning_rate: 0.0000e+00
Epoch 4/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12113s 3s/step - accuracy: 0.3288 - loss: 2.1979 - val_accuracy: 0.3812 - val_loss: 2.1411 - learning_rate: 0.0000e+00
Epoch 5/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 12072s 3s/step - accuracy: 0.3298 - loss: 2.1966 - val_accuracy: 0.3812 - val_loss: 2.1411 - learning_rate: 0.0000e+00
Epoch 6/15
 993/4276 ━━━━━━━━━━━━━━━━━━━━ 2:26:21 3s/step - accuracy: 0.3264 - loss: 2.2104

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

# Classification report and confusion matrix
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

print(classification_report(y_true, y_pred))



266/266 ━━━━━━━━━━━━━━━━━━━━ 775s 3s/step - accuracy: 0.5239 - loss: 1.6376
Test Accuracy: 0.52
266/266 ━━━━━━━━━━━━━━━━━━━━ 722s 3s/step
              precision    recall  f1-score   support

           0       0.49      0.45      0.47       311
           1       0.46      0.52      0.49       267
           2       0.78      0.86      0.82       395
           3       0.46      0.08      0.13        77
           4       0.35      0.37      0.36       487
           5       0.37      0.21      0.27       207
           6       0.52      0.63      0.57       505
           7       0.53      0.20      0.29        81
           8       0.32      0.18      0.23       249
           9       0.37      0.33      0.35       324
          10       0.51      0.40      0.45       507
          11       0.62      0.76      0.68       431
          12       0.57      0.30      0.39        80
          13       0.43      0.41      0.42       499
          14       0.44      0.09      0.14        

In [ ]:
print(confusion_matrix(y_true, y_pred))

[[139   2   6   0   4   0   1   0   0   0   0   1   1   2   0  61  22   1
    7   1   4   0  49   3   7   0   0]
 [  3 139   8   2  34   2   1   0   2   8   2   3   1   6   0  13   7   2
    3   0  17   2   2   2   8   0   0]
 [  3   6 340   0   3   2   3   0   0   0   0   1   0   0   0  14  11   0
    1   1   1   0   2   1   5   0   1]
 [  2   8   4   6   6   4   4   0   0   3   0   2   1   6   0   9   2   0
    7   0   7   1   2   2   1   0   0]
 [  3  39   4   2 178   6  79   0  12  29   9  11   0  30   1   7   4   1
   15   5  38   4   1   0   4   4   1]
 [  3   8  11   1  36  43   7   1   5   3   2   2   3   9   0  11  20   2
   10   0   9   0   3   5  11   0   2]
 [  2   8   2   0  24   1 320   0   8   5   5   6   0   8   0   1   4   1
   10   5  64  13   2   3   0   8   5]
 [  0   2   1   0   9   3   2  16   1   3   1   8   0   4   0   0   1   0
   10   2  12   3   0   0   3   0   0]
 [  0  12   1   0  44   3  22   0  46  13  10   9   1  16   3   0   1   0
    4   9  39   8   2 

In [ ]:
model.save('my_model_VGG16_lr_1e-6.keras')

In [ ]:
import json

# Save class indices from the train_generator
with open('class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f)

print("Class indices saved successfully!")

Class indices saved successfully!


In [ ]:
# === Step 8: Evaluate the Model ===
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

# Classification report and confusion matrix
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

266/266 ━━━━━━━━━━━━━━━━━━━━ 381s 1s/step - accuracy: 0.4058 - loss: 2.0286
Test Accuracy: 0.41
266/266 ━━━━━━━━━━━━━━━━━━━━ 371s 1s/step
              precision    recall  f1-score   support

           0       0.41      0.50      0.45       311
           1       0.38      0.33      0.35       267
           2       0.69      0.72      0.70       395
           3       1.00      0.04      0.07        77
           4       0.24      0.19      0.22       487
           5       0.25      0.05      0.09       207
           6       0.37      0.47      0.41       505
           7       0.67      0.02      0.05        81
           8       0.19      0.07      0.10       249
           9       0.27      0.16      0.20       324
          10       0.38      0.38      0.38       507
          11       0.42      0.69      0.52       431
          12       0.33      0.10      0.15        80
          13       0.33      0.30      0.31       499
          14       0.00      0.00      0.00        

In [ ]:
# Get the updated learning rate from the optimizer
current_learning_rate = model.optimizer.learning_rate.numpy()
print(f"Current Learning Rate: {current_learning_rate}")



Current Learning Rate: 2.499999936844688e-05


Image augmentation

In [ ]:
model.save('my_model.keras')


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min=1e-6, verbose=1)

In [ ]:
# === Step 7: Fine-Tune the Model ===
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=2.5e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks = [reduce_lr]
)


Epoch 1/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7348s 2s/step - accuracy: 0.3710 - loss: 2.0292 - val_accuracy: 0.4029 - val_loss: 2.0702 - learning_rate: 2.5000e-05
Epoch 2/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7379s 2s/step - accuracy: 0.3717 - loss: 2.0209 - val_accuracy: 0.4109 - val_loss: 2.0192 - learning_rate: 2.5000e-05
Epoch 3/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7519s 2s/step - accuracy: 0.3761 - loss: 2.0125 - val_accuracy: 0.4057 - val_loss: 2.0449 - learning_rate: 2.5000e-05
Epoch 4/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3769 - loss: 2.0111
Epoch 4: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7384s 2s/step - accuracy: 0.3769 - loss: 2.0111 - val_accuracy: 0.3976 - val_loss: 2.0753 - learning_rate: 2.5000e-05
Epoch 5/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7365s 2s/step - accuracy: 0.3823 - loss: 1.9882 - val_accuracy: 0.4146 - val_loss: 2.0243 - learning_rate: 1.2500e-05
Epoch 6/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7412s 

In [ ]:
model.save('my_model_3e-6.keras')

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min=1e-6, verbose=1)

In [ ]:
# === Step 7: Fine-Tune the Model ===
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=3.125e-6), loss='categorical_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks = [reduce_lr]
)

Epoch 1/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7815s 2s/step - accuracy: 0.3944 - loss: 1.9436 - val_accuracy: 0.4356 - val_loss: 1.9582 - learning_rate: 3.1250e-06
Epoch 2/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7690s 2s/step - accuracy: 0.3964 - loss: 1.9425 - val_accuracy: 0.4315 - val_loss: 1.9613 - learning_rate: 3.1250e-06
Epoch 3/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3971 - loss: 1.9369
Epoch 3: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-06.
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7784s 2s/step - accuracy: 0.3971 - loss: 1.9369 - val_accuracy: 0.4328 - val_loss: 1.9658 - learning_rate: 3.1250e-06
Epoch 4/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7723s 2s/step - accuracy: 0.3961 - loss: 1.9407 - val_accuracy: 0.4333 - val_loss: 1.9615 - learning_rate: 1.5625e-06
Epoch 5/15
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3964 - loss: 1.9374
Epoch 5: ReduceLROnPlateau reducing learning rate to 7.81249980263965e-07.
4276/4276 ━━━━━━━━━━━━━━━━━━━━ 7771s 2s/step - 

In [ ]:
model.save('my_model_1_95e-6.keras')